# Revisando os Conceitos da Aula Anterior


## ETL e ELT


#### ETL -> Extract Transform and Load

- Dados ingeridos já de acordo com sua finalidade
- Sempre que adquiro um dado já aplico as transformações de negócio necessárias
- Necessita de um conhecimento Prévio da Finalidade do Dado

#### ELT -> Extract Load and Transform

- Todo dado é válido
- Armazenamento do dado em seu estado original
- Não necessitamos conhecer a finalidade para a aquisição de um dado
- Torna mais flexível e reaproveitável o uso do dado ingerido
- Permite níveis de refinamento diferentes do dado disponíveis no ambiente


## APIs

- Biblioteca Python Requests

  - > %pip install requests
  - > import requests

- APIs são interfaces que nos permitem conversar com outros sistemas e contextos

- Principais verbos HTTP

  - GET
  - POST
  - PUT
  - PATCH
  - DELETE

- Introdução aos Códigos de Resposta HTTP
  - 200 - 299 - OK
  - 400 - 499 - Erro por parte do Cliente
  - 500 - 599 - Erro por parte do Servidor


### Autenticação

Principais Tipos de Autenticação

- Baerer Token


In [ ]:
import requests
import json

url_base = "https://dummyjson.com/auth/"


def authenticator(login, senha):
    login_response = requests.post(
        url="https://dummyjson.com/auth/login",
        headers={'Content-Type': 'application/json'},
        data=json.dumps({
            "username": login,
            "password": senha
        })
    )
    return login_response.json()['token']


access_token = authenticator(login="kminchelle", senha="0lelplR")

authorization_header = {
    "Authorization": f"Bearer {access_token}",
    'Content-Type': 'application/json'
}

authenticated_products = requests.get(
    url=f"{url_base}/products",
    headers=authorization_header
)

authenticated_products.status_code

# Implementação Re-autenticação


In [7]:
def created_authentication_headers(url: str):
    access_token = authenticator(login="kminchelle", senha="0lelplR")

    return {
        "Authorization": f"Bearer {access_token}",
        'Content-Type': 'application/json'
    }

In [6]:
def management_refresh_token(url: str, authorization_header, current_retry: int = 0, max_retries: int = 0):

    response = requests.get(
        url=url,
        headers=authorization_header
    )

    try:
        if response.status_code == 401 and current_retry < max_retries and response.json()['name'] == "TokenExpiredError":
            return management_refresh_token(
                url=url,
                authorization_header=created_authentication_headers()
            )
        else:
            print(
                f"Não foi possivel recuperar após o código. Código: {response.status_code}")

    except Exception as ex:
        print(f"Fluxo de exceção. Detalhe: {ex}")

    return